In [1]:
import numpy as np

from scipy.stats import norm
from tqdm import tqdm
import os

from Chempy.parameter import ModelParameters

import sbi.utils as utils

import torch
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform

import time as t
import pickle

import multiprocessing as mp
import matplotlib.pyplot as plt

from plot_functions import *

In [2]:
# ----- Load the data -----
a = ModelParameters()
labels = [a.to_optimize[i] for i in range(len(a.to_optimize))] + ['time']
priors = torch.tensor([[a.priors[opt][0], a.priors[opt][1]] for opt in a.to_optimize])

elements = a.elements_to_trace

In [3]:
ape = torch.load('data/ape_sbi_w5p-error_noH.pt')

/tmp/ipykernel_701510/505595515.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ape = torch.load('data/ape_sbi_w5p-error_noH.pt')


In [4]:
ape.shape

torch.Size([45771, 6])

In [5]:
ape_plot(ape, labels, os.path.join(os.getcwd(), 'ape_test.png'))


APE of the Posterior:
Median + upper quantile - lower quantile
Total : 13.7% + 22.1 - 10.6

high_mass_slope : 1.9% + 1.8 - 1.0
log10_N_0 : 2.6% + 2.5 - 1.5
log10_starformation_efficiency : 71.0% + 90.1 - 31.5
log10_sfr_scale : 19.6% + 12.1 - 9.9
outflow_feedback_fraction : 24.4% + 17.7 - 12.6
time : 19.4% + 12.8 - 9.2



<Figure size 640x480 with 0 Axes>

In [ ]:
ape.flatten().shape

In [ ]:
np.percentile(ape, [25, 50, 75])

In [ ]:
ape.T[:,:1000].shape

In [ ]:
ape.shape

In [ ]:
plt.boxplot(ape, vert=False, autorange=False, widths=0.5, patch_artist=True, showfliers=False)

In [ ]:
fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.20, .80)})
colors = ["tomato", "skyblue", "olive", "gold", "teal", "orchid"]

print("APE of the posterior:")
print("Median + upper quantile - lower quantile")
l_quantile, median, u_quantile = np.percentile(ape, [25, 50, 75])
print(f"Total : {median:.1f}% + {u_quantile-median:.1f} - {median-l_quantile:.1f}")
print("")

for i in range(ape.shape[1]):
    l_quantile, median, u_quantile = np.percentile(ape[:,i], [25, 50, 75])
    ax_hist.hist(ape[:,i], bins=25, density=True, range=(0, 100), label=labels[i], color=colors[i], alpha=0.5)
    median = np.percentile(ape[:,i], 50)
    ax_hist.axvline(median, color=colors[i], linestyle='--')
    #print(labels[i] + fr' ${median:.1f}^{{+{u_quantile-median:.1f}}}_{{-{median-l_quantile:.1f}}}\%$')
    print(labels[i] + f" : {median:.1f}% + {u_quantile-median:.1f} - {median-l_quantile:.1f}")
    
ax_hist.set_xlabel('Error (%)', fontsize=15)
ax_hist.set_ylabel('Density', fontsize=15)
ax_hist.spines['top'].set_visible(False)
ax_hist.spines['right'].set_visible(False)
ax_hist.legend()

bplot = ax_box.boxplot(ape, vert=False, autorange=False, widths=0.5, patch_artist=True, showfliers=False, boxprops=dict(facecolor='tomato'), medianprops=dict(color='black'))
#plt.style.use('dark_background')
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
ax_box.set(yticks=[])
ax_box.spines['left'].set_visible(False)
ax_box.spines['right'].set_visible(False)
ax_box.spines['top'].set_visible(False)

fig.suptitle('APE of the Posterior', fontsize=20)
plt.xlim(0, 100)
fig.tight_layout()
plt.show()

In [ ]:
ape.T